<a href="https://colab.research.google.com/github/divyansh1195/Kaggle-ML-Exercise-Notebooks/blob/master/Intermediate_ML_Missing_values.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Intermediate ML:Missing values**

In [ ]:
import pandas as pd

# Read the data
train_data = pd.read_csv('train.csv', index_col='Id')
test_data = pd.read_csv('test.csv', index_col='Id')
#print(train_data)
#print(test_data)

In [ ]:
# Remove rows with missing target values
train_data.dropna(axis=0, subset=['SalePrice'], inplace=True)
#axis=0 or, 'index' drops rows with missing values, axis=1 or, 'columns'drops columns which contain missing value
#subset label along which you wish to drop

#define y after removing missing values
y = train_data.SalePrice
print(y)

#separate target from predictors
#here, separate SalePrice from reamining columns
train_data.drop(['SalePrice'], axis=1, inplace=True)
#SalePrice column removed from X_full

# To keep things simple, we'll use only numerical predictors
X = train_data.select_dtypes(exclude=['object'])
test = test_data.select_dtypes(exclude=['object'])

print(X)
#print(test)

Id
1       208500
2       181500
3       223500
4       140000
5       250000
         ...  
1456    175000
1457    210000
1458    266500
1459    142125
1460    147500
Name: SalePrice, Length: 1460, dtype: int64
      MSSubClass  LotFrontage  LotArea  ...  MiscVal  MoSold  YrSold
Id                                      ...                         
1             60         65.0     8450  ...        0       2    2008
2             20         80.0     9600  ...        0       5    2007
3             60         68.0    11250  ...        0       9    2008
4             70         60.0     9550  ...        0       2    2006
5             60         84.0    14260  ...        0      12    2008
...          ...          ...      ...  ...      ...     ...     ...
1456          60         62.0     7917  ...        0       8    2007
1457          20         85.0    13175  ...        0       2    2010
1458          70         66.0     9042  ...     2500       5    2010
1459          20         68.0

In [ ]:
# Break off test set from training data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [ ]:
X_train.head(2)

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
619,20,90.0,11694,9,5,2007,2007,452.0,48,0,1774,1822,1828,0,0,1828,0,0,2,0,3,1,9,1,2007.0,3,774,0,108,0,0,260,0,0,7,2007
871,20,60.0,6600,5,5,1962,1962,0.0,0,0,894,894,894,0,0,894,0,0,1,0,2,1,5,0,1962.0,1,308,0,0,0,0,0,0,0,8,2009


In [ ]:
# Shape of training data (num_rows, num_columns)
print(X_train.shape)
#print(X_train.columns)
# Number of missing values in each column of training data
missing_val_count_by_column = (X_train.isnull().sum())
#print(missing_val_count_by_column)
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(1168, 36)
LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64


Best approach to dealing with missing values

Approach 1 (Drop Columns with Missing Values)                   **(Usually this is a wrong approach)**

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_test, y_train, y_test):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    return mean_absolute_error(y_test, preds)

In [ ]:
# Get names of columns with missing values
cols_with_missing = [col for col in X_train.columns
                     if X_train[col].isnull().any()]

# Drop columns in training and validation data
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_test = X_test.drop(cols_with_missing, axis=1)

print("MAE from Approach 1 (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_test, y_train, y_test))
print(reduced_X_train.shape)

MAE from Approach 1 (Drop columns with missing values):
17837.82570776256
(1168, 33)


Approach 2 (Imputation)
SimpleImputer to replace missing values with the mean value along each column

In [ ]:
from sklearn.impute import SimpleImputer

# Imputation
imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(imputer.fit_transform(X_train))
imputed_X_test = pd.DataFrame(imputer.transform(X_test))

# Imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_test.columns = X_test.columns

print("MAE from Approach 2 (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_test, y_train, y_test))

#check if any missing value in any column
missing_val_count_by_column = (imputed_X_train.isnull().sum())
print(missing_val_count_by_column)
#print(missing_val_count_by_column[missing_val_count_by_column > 0])

MAE from Approach 2 (Imputation):
18062.894611872147
MSSubClass       0
LotFrontage      0
LotArea          0
OverallQual      0
OverallCond      0
YearBuilt        0
YearRemodAdd     0
MasVnrArea       0
BsmtFinSF1       0
BsmtFinSF2       0
BsmtUnfSF        0
TotalBsmtSF      0
1stFlrSF         0
2ndFlrSF         0
LowQualFinSF     0
GrLivArea        0
BsmtFullBath     0
BsmtHalfBath     0
FullBath         0
HalfBath         0
BedroomAbvGr     0
KitchenAbvGr     0
TotRmsAbvGrd     0
Fireplaces       0
GarageYrBlt      0
GarageCars       0
GarageArea       0
WoodDeckSF       0
OpenPorchSF      0
EnclosedPorch    0
3SsnPorch        0
ScreenPorch      0
PoolArea         0
MiscVal          0
MoSold           0
YrSold           0
dtype: int64


Preprocess the training and validation data using Imputer MEDIAN strategy

In [ ]:
final_imputer = SimpleImputer(strategy='median')
final_X_train = pd.DataFrame(final_imputer.fit_transform(X_train))
final_X_test = pd.DataFrame(final_imputer.transform(X_test))

# Imputation removed column names; put them back
final_X_train.columns = X_train.columns
final_X_test.columns = X_test.columns

missing_val_count_by_column = (final_X_train.isnull().sum())
print(missing_val_count_by_column)
#print(missing_val_count_by_column[missing_val_count_by_column > 0])

MSSubClass       0
LotFrontage      0
LotArea          0
OverallQual      0
OverallCond      0
YearBuilt        0
YearRemodAdd     0
MasVnrArea       0
BsmtFinSF1       0
BsmtFinSF2       0
BsmtUnfSF        0
TotalBsmtSF      0
1stFlrSF         0
2ndFlrSF         0
LowQualFinSF     0
GrLivArea        0
BsmtFullBath     0
BsmtHalfBath     0
FullBath         0
HalfBath         0
BedroomAbvGr     0
KitchenAbvGr     0
TotRmsAbvGrd     0
Fireplaces       0
GarageYrBlt      0
GarageCars       0
GarageArea       0
WoodDeckSF       0
OpenPorchSF      0
EnclosedPorch    0
3SsnPorch        0
ScreenPorch      0
PoolArea         0
MiscVal          0
MoSold           0
YrSold           0
dtype: int64


In [ ]:
#Evaluate RANDOM FOREST REGRESSION model

# Define and fit model

model = RandomForestRegressor(n_estimators=150, random_state=0)
model.fit(final_X_train, y_train)

# Get validation predictions and MAE
preds_test = model.predict(final_X_test)
print("MAE (Your approach):")
print(mean_absolute_error(y_test, preds_test))

MAE (Your approach):
17576.83299847793


Applying on 'test' dataset

In [ ]:
final_test = pd.DataFrame(final_imputer.fit_transform(test))

# get test predictions
preds_test = model.predict(final_test)
print(preds_test)
#predictor values of houses for 'test' based on 'train'

[127353.72       154886.66666667 181673.82666667 ... 157433.36
 108949.         231465.03333333]


In [ ]:
output = pd.DataFrame({'Id': test.index,
                       'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)
z=pd.read_csv('submission.csv')
z.shape

(1459, 2)